In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

#leggo da file la matrice di copertura 
mod = gp.Model('WDCP')

file = open('TEST2.txt', 'r')
content = file.readlines()
A = []
for i in content:
    array = []
    parse = i.split(",")
    for j in parse:
        array.append(float(j))
    A.append(array)
file.close()
A = np.array(A)




file2 = open('TEST2sj.txt', 'r')
content = file2.readline()
pars1=content.split(",")
s = []
for i in pars1:
    s.append(float(i))
file.close() 
s = np.array(s)  
    


numero_righe=len(A)
numero_colonne=len(A[0])
        
    
print('A: ',A)
print('righe di A: ',numero_righe)
print('colonne di A: ',numero_colonne)
print('s: ',s,len(s))

if (len(s)!=numero_colonne):
    print('errore')
    
    


Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-13
A:  [[1. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 1. 1.]]
righe di A:  18
colonne di A:  121
s:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.] 121


In [10]:
#parametro alpha per i punti generali
alpha=0.21;

#creazione del modello
mod = gp.Model('WDCP')
L = range(numero_righe)
R = range(numero_colonne)
y = mod.addVars(L,vtype=GRB.BINARY,name="y")
x = mod.addVars(R,vtype=GRB.BINARY,name="x")


# Creazione dell'obiettivo
obj = mod.setObjective(gp.quicksum(y[i] for i in L) - alpha * gp.quicksum((1-s[j])*x[j] for j in R), GRB.MINIMIZE)


#vincoli
for j in R:  
    if s[j]==1: #aggiungo per forza i punti importanti
        lu=1;
        mod.addConstr(gp.quicksum(A[i,j]*y[i] for i in L) >= 1)
    else:     #altrimenti, il punto è generale e la sua appartenza alla soluzione dipende da alpha
        mod.addConstr(gp.quicksum(A[i,j] * y[i] for i in L) >= x[j]) 
    for i in L:
        mod.addConstr((A[i,j]*y[i])<=x[j])  #se posiziono la telecamera devo segnare i punti coperti

# Risoluzione del problema
mod.optimize()

# Stampa dei risultati

for i in L:
    print(f"y_{i+1} =", int(y[i].x))

    sum=0
for j in R:
    print(f"x_{j+1} =", int(x[j].x))
    sum=sum+int(x[j].x)
print(sum) #punti coperti





Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700HQ CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2299 rows, 139 columns and 2966 nonzeros
Model fingerprint: 0x317a15c1
Variable types: 0 continuous, 139 integer (139 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -12.3600000
Presolve removed 2269 rows and 98 columns
Presolve time: 0.00s
Presolved: 30 rows, 41 columns, 131 nonzeros
Found heuristic solution: objective -15.1000000
Variable types: 0 continuous, 41 integer (41 binary)

Root relaxation: objective -1.552000e+01, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Bes

In [7]:
puntivisti=0;
    
for j in R:
    puntivisti=puntivisti+int(x[j].x)
    #if (x[j].x==1):
       # print("visti: x",j) 

puntiimportanti=0;
    
for j in range(numero_colonne):
    puntiimportanti=puntiimportanti+s[j]

telecameremesse=0;
for i in L:
    telecameremesse=telecameremesse+int(y[i].x)
    
print("numero di posizioni osservate: ",puntivisti) #punti coperti
print("numero telecamere posizionate: ",telecameremesse) 
print("numero punti importanti: ",puntiimportanti) 



numero di posizioni osservate:  112
numero telecamere posizionate:  8
numero punti importanti:  5.0


In [8]:


#creazione del modello
mod = gp.Model('WDCP')
L = range(numero_righe)
R = range(numero_colonne)
y = mod.addVars(L,vtype=GRB.BINARY,name="y")
x = mod.addVars(R,vtype=GRB.BINARY,name="x")


# Creazione dell'obiettivo
obj = mod.setObjective(gp.quicksum(y[i] for i in L) - alpha * gp.quicksum((1-s[j])*x[j] for j in R), GRB.MINIMIZE)


#vincoli
for j in R:  
    if s[j]==1: #aggiungo per forza i punti importanti
        lu=1;
        #mod.addConstr(gp.quicksum(A[i,j]*y[i] for i in L) >= 1)
    else:     #altrimenti, il punto è generale e la sua appartenza alla soluzione dipende da alpha
        mod.addConstr(gp.quicksum(A[i,j] * y[i] for i in L) >= x[j]) 
    for i in L:
        mod.addConstr((A[i,j]*y[i])<=x[j])  #se posiziono la telecamera devo segnare i punti coperti

# Risoluzione del problema
mod.optimize()

# Stampa dei risultati

for i in L:
    print(f"y_{i+1} =", int(y[i].x))

    sum=0
for j in R:
    print(f"x_{j+1} =", int(x[j].x))
    sum=sum+int(x[j].x)
print(sum) #punti coperti



Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700HQ CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2294 rows, 139 columns and 2958 nonzeros
Model fingerprint: 0x171171c9
Variable types: 0 continuous, 139 integer (139 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000
Found heuristic solution: objective -12.3600000
Presolve removed 2259 rows and 92 columns
Presolve time: 0.01s
Presolved: 35 rows, 47 columns, 157 nonzeros
Found heuristic solution: objective -15.1000000
Variable types: 0 continuous, 47 integer (47 binary)

Root relaxation: objective -1.552000e+01, 9 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl 

In [9]:
puntivisti=0;
    
for j in R:
    puntivisti=puntivisti+int(x[j].x)
    #if (x[j].x==1):
       # print("visti: x",j) 

puntiimportanti=0;
    
for j in range(numero_colonne):
    puntiimportanti=puntiimportanti+s[j]

telecameremesse=0;
for i in L:
    telecameremesse=telecameremesse+int(y[i].x)
print("LB")
print("numero di posizioni osservate: ",puntivisti) #punti coperti
print("numero telecamere posizionate: ",telecameremesse) 
print("numero punti importanti: ",puntiimportanti) 


LB
numero di posizioni osservate:  117
numero telecamere posizionate:  8
numero punti importanti:  5.0
